In [18]:
# connector de python à mysql
import mysql.connector
import json
#**************************************************************************************CREATION DE LA CONNEXION SQLALCHEMY**********************************************************

## IMPORT DU CONFIG.JSON
# assignation de la config.json à fichierConfig
fichierConfig = "config.json"
# ouverture et chargement des donnée contenu dans fichierConfig
with open(fichierConfig) as fichier:
    config = json.load(fichier)["mysql"]

# assignation de de la connexion par create_engine avec les éléement de connexion + les info du fichierConfig à engine
# en dehors de la class car cet élémeent ne change pas
connection = mysql.connector.connect(host=config["host"],database = config["bdd"],user=config["user"],password=config["password"])
cursor = connection.cursor()

In [222]:
## ON CHECK SI UNE RESERVATION EXISTE DEJA DANS LA BASE DE DONNEE POUR LE MEME "nom de salle" la même "DATE" et si les heures peuvent correspondre

num_salle_reserv = "736"
date_reserv = "2021-01-17"
debut_reserv = 12
fin_reserv = 15

In [223]:
cursor.execute('SELECT res_id, res_date, res_heure_arrive, res_heure_depart, S.sa_name FROM RESERVATION R INNER JOIN SALLE S ON S.sa_id=R.sa_id WHERE S.sa_name=%s AND R.res_date=%s AND(( %s BETWEEN R.res_heure_arrive AND R.res_heure_depart ) OR (%s BETWEEN R.res_heure_arrive AND R.res_heure_depart));', (num_salle_reserv,date_reserv,debut_reserv,fin_reserv))

resa_db_1 = cursor.fetchone()

#print(resa_db_1)

if resa_db_1 == None:
    result = True,"TESTING OK FIRST"
    print(result)

###################################################### TEST DU SI L'HEURE  DE DEBUT DEMANDER CORRESPOND A UNE HEURE DE FIN DEJA EXISTANTE#####################
elif resa_db_1 != None and debut_reserv == resa_db_1[3]:
    cursor.execute('SELECT * FROM RESERVATION R INNER JOIN SALLE S ON S.sa_id=R.sa_id WHERE S.sa_name=%s AND R.res_date=%s AND ((%s > R.res_heure_arrive AND %s < R.res_heure_depart) OR (%s >= R.res_heure_arrive AND %s <= R.res_heure_depart));', (num_salle_reserv,date_reserv,debut_reserv,debut_reserv, fin_reserv,fin_reserv))
    result_fin_to_debut_reserv = cursor.fetchone()
    
    if result_fin_to_debut_reserv == None and resa_db_1 != None:
        result = True,"TESTING IS OK 1"
        print(result)
    


########################################################### TEST DU SI L'HEURE DE FIN DEMANDER CORRESPOND A UNE HEURE DE DEBUT DEJA EXISTANTE#################

elif resa_db_1 != None and fin_reserv == resa_db_1[2]:
    cursor.execute('SELECT * FROM RESERVATION R INNER JOIN SALLE S ON S.sa_id=R.sa_id WHERE S.sa_name=%s AND R.res_date=%s AND ((%s >= R.res_heure_arrive AND %s <= R.res_heure_depart) OR (%s > R.res_heure_arrive AND %s < R.res_heure_depart));', (num_salle_reserv,date_reserv,debut_reserv,debut_reserv, fin_reserv,fin_reserv))
    result_debut_to_fin_reserv = cursor.fetchone()
    
    if result_debut_to_fin_reserv == None and resa_db_1 != None:
        result = True,"TESTING IS OK 2"
        print(result)


else:
    result_fin_to_debut_reserv != None and resa_db_1 != None or result_debut_to_fin_reserv != None and resa_db_1 != None
    result = False,"TESTING NOT OK 1"
    print(result)


(True, 'TESTING OK FIRST')
